In [47]:
import csv
import os
import avro.schema
from avro.datafile import DataFileWriter
from avro.io import DatumWriter
from collections import namedtuple
import logging
from logging.config import fileConfig

#fileConfig('logging_config.ini')
log = logging.getLogger()


fields = ("area_fips",
         "own_code",
         "industry_code",
         "agglvl_code",
         "size_code",
         "year",
         "qtr",
         "disclosure_code",
         "area_title",
         "own_title",
         "industry_title",
         "agglvl_title",
         "size_title",
         "qtrly_estabs",
         "month1_emplvl",
         "month2_emplvl",
         "month3_emplvl",
         "total_qtrly_wages",
         "taxable_qtrly_wages",
         "qtrly_contributions",
         "avg_wkly_wage",
         "lq_disclosure_code",
         "lq_qtrly_estabs",
         "lq_month1_emplvl",
         "lq_month2_emplvl",
         "lq_month3_emplvl",
         "lq_total_qtrly_wages",
         "lq_taxable_qtrly_wages",
         "lq_qtrly_contributions",
         "lq_avg_wkly_wage",
         "oty_disclosure_code",
         "oty_qtrly_estabs_chg",
         "oty_qtrly_estabs_pct_chg",
         "oty_month1_emplvl_chg",
         "oty_month1_emplvl_pct_chg",
         "oty_month2_emplvl_chg",
         "oty_month2_emplvl_pct_chg",
         "oty_month3_emplvl_chg",
         "oty_month3_emplvl_pct_chg",
         "oty_total_qtrly_wages_chg",
         "oty_total_qtrly_wages_pct_chg",
         "oty_taxable_qtrly_wages_chg",
         "oty_taxable_qtrly_wages_pct_chg",
         "oty_qtrly_contributions_chg",
         "oty_qtrly_contributions_pct_chg",
         "oty_avg_wkly_wage_chg",
         "oty_avg_wkly_wage_pct_chg")


DataRecord = namedtuple('DataRecord', fields)

            
def read_data(path):
    with open(path, 'rU') as data:
        data.readline()                          # skip the header
        reader = csv.reader(data)
        for row in map(DataRecord._make, reader):
            yield row
            
def parse_schema(path="data/qcew_schema.avsc"):
    with open(path, 'r') as data:
        return avro.schema.parse(data.read())
    
def serialize_records(records, outpath = "qcew_data.avro"):
    schema = parse_schema()
    
    # Check if file exists, it not make one
    # Need to handle cases where the file exists and we may not want to append
    if os.path.isfile("qcew_data.avro") is None:
        open("qcew_data.avro", 'a').close()

    with open(outpath, 'a') as out:
        writer = DataFileWriter(out, DatumWriter(), schema)
        for record in records:
            record = dict((f, getattr(record, f)) for f in record._fields)
            writer.append(record)


def iterate_serialize():
    # This is pretty hacky
    dirs = [d for d in os.listdir('data') if os.path.isdir(os.path.join('data', d))]
    for folder in dirs:

        folder_path = os.path.join('data', folder)
        files = os.listdir(folder_path)
        # need to add in loggin to track what file I am in and setup recovery

        for file in files:
            filepath = os.path.join(folder_path, file)
            #print os.path.join('data', folder, filepath) + "\n"
            log.warning("SERIALIZING: %s" % filepath)
            reader = read_data(filepath)
            serialize_records(reader)

    
if __name__ == "__main__":
    iterate_serialize()

KeyboardInterrupt: 

In [37]:
# This is pretty hacky
dirs = [d for d in os.listdir('data') if os.path.isdir(os.path.join('data', d))]
for folder in dirs:
    
    folder_path = os.path.join('data', folder)
    files = os.listdir(folder_path)
    # need to add in loggin to track what file I am in and setup recovery
    
    for file in files:
        filepath = folder + file
        print os.path.join('data', folder, filepath) + "\n"

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 921150 Tribal governments.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 92119 Other general government support.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 921190 Other general government support.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 922 Justice, public order, and safety activities.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 9221 Justice, public order, and safety activities.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 92211 Courts.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 922110 Courts.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 92212 Police protection.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 922120 Police protection.csv

data/2013.q1-q4.by_industry/2013.q1-q4.by_industry2013.q1-q4 92213 Legal counsel and prosecution.csv

data/2013.q1-q4.by_indus